# SwissPollenTools | Inference Pipeline Example

1. Loading the tools
2. Creating the pipeline configuration
3. Creating the pipeline
4. Running the inference pipeline

## 1. Loading the tools

In [ ]:
from swisspollentools.pipelines import InferencePipelineConfig, InferencePipeline

### Loading the Keras Model
> Note that implementing a new model, it is preferable to implement the respective postprocessing function and to provide it with the model.

For demonstration purposes, we implement a random model and we design a postprocessing function associated with it. 
In an operational setup, the model would be loaded using the `keras.models.load_model` command and the post processing function would be implemented (copied) from the inference script.

In [ ]:
import numpy as np

class RandomModel():
    def __init__(self):
        pass

    def predict(self, batch, *args, **kwargs):
        return np.random.random((len(batch["rec0"]), 8))
    
def post_processing_fn(batch):
    predicted_class = np.argmax(batch, axis=-1)
    predicted_certainity = np.max(batch, axis=-1)
    return {
        "class": predicted_class,
        "certainity": predicted_certainity
    }

## 2. Creating the pipeline configuration

### Extraction Tool Parameters
- `exw_batch_size`
- `exw_keep_metadata`
- `exw_keep_fluorescence`
- `exw_keep_rec_properties`
- `exw_keep_metadata_key`
- `exw_keep_fluorescence_keys`
- `exw_keep_rec_properties_keys`
- `exw_filters`
### Inference Tool Parameters
- `inw_from_rec0`
- `inw_from_rec1`
- `inw_from_fluorescence`
- `inw_from_fluorescence_keys`
- `inw_rec_shape`
- `inw_rec_precision`
- `inw_batch_size`
- `inw_post_processing_fn`
### ToCSV Tool Parameters
- `tocsvw_output_directory`

In [ ]:
config = InferencePipelineConfig(
    exw_batch_size=1024,
    exw_keep_metadata_key=["eventId"],
    exw_keep_fluorescence_keys=["average_std", "average_mean", "relative_spectra"],
    exw_filters={"max_area": 625, "max_solidity": 0.9},
    inw_from_fluorescence=False,
    inw_batch_size=256,
    inw_post_processing_fn=post_processing_fn,
    tocsvw_output_directory="./tmp"
)

## 3. Creating the pipeline

In [ ]:
pipeline = InferencePipeline(config, inw_model=RandomModel())

## 4. Running the inference pipeline

In [ ]:
import tensorflow as tf 

with tf.device("/cpu:0"):
    out = pipeline(file_path="./path/to/example.zip")